In [149]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [150]:
# https://github.com/Protean-Labs/subgrounds
from subgrounds.subgrounds import Subgrounds
from subgrounds.subgraph import SyntheticField
from datetime import datetime

sg = Subgrounds()

In [151]:
# SushiSwap theGraph node 1
sushi_1 = sg.load_subgraph('https://api.thegraph.com/subgraphs/name/sushiswap/exchange')
# SushiSwap theGraph node 2 from dev docs
sushi_2 = sg.load_subgraph('https://api.thegraph.com/subgraphs/name/zippoxer/sushiswap-subgraph-fork')

In [152]:
# LP snapshots containing provider, amount and timestamp
liqPosSnaps = sushi_1.Query.liquidityPositionSnapshots(
    first = 1000,
    orderBy = 'timestamp',
    orderDirection = 'desc',
    where = {
        'liquidityTokenBalance_gt': 0
    }
)

# LP pairs
pairs = sushi_1.Query.pairs(
    first = 3000
)

## Synthetic field to transfer timestamp to datetime
sushi_1.LiquidityPositionSnapshot.datetime = SyntheticField(
    f = lambda timestamp: str(datetime.fromtimestamp(timestamp)),
    type_ = SyntheticField.STRING,
    deps = sushi_1.LiquidityPositionSnapshot.timestamp
)

In [153]:
[pairs, lp_snaps] = sg.query_df([ 
    pairs.id,
    pairs.name,
    liqPosSnaps.datetime,
    # liqPosSnaps.timestamp,
    liqPosSnaps.user.id,
    liqPosSnaps.pair.id,
    liqPosSnaps.liquidityTokenBalance
])

pairs.columns = ['id','name']
lp_snaps.columns = ['datetime','user_id','pair_id','lp_token_balance']

sushi_df = pd.merge(
        lp_snaps, 
        pairs, 
        how='left', 
        left_on='pair_id', 
        right_on='id'
    )\
    .drop(
        labels=['id'],
        axis=1
    )


In [155]:
sushi_df.head()

,datetime,user_id,pair_id,lp_token_balance,name
0,2022-04-03 19:59:49,0x5ad6211cd3fde39a9cecb5df6f380b8263d1e277,0xaf491d603db3c1138a5ee5db35eb0c7b6c4541a2,347.162593,PLAY-DOUGH
1,2022-04-03 19:59:01,0x2e54c2b2982a4ebc5522b4826518da2a7eb5bfdc,0xeefa3b448768dd561af4f743c9e925987a1f8d09,132522.564114,WETH-NFD
2,2022-04-03 19:58:59,0xaf2fe0d4fe879066b2baa68d9e56cc375df22815,0xaf491d603db3c1138a5ee5db35eb0c7b6c4541a2,23160.598051,PLAY-DOUGH
3,2022-04-03 19:58:59,0x79939118ef2af99e20310ecc5ccc0e9697d390fc,0xaf491d603db3c1138a5ee5db35eb0c7b6c4541a2,9950.000000,PLAY-DOUGH
4,2022-04-03 19:58:59,0x6de77a304609472a4811a0bfd47d8682aebc29df,0xaf491d603db3c1138a5ee5db35eb0c7b6c4541a2,526362.965587,PLAY-DOUGH
